In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, Concatenate, add, BatchNormalization, Activation
import os
import glob
import tifffile as tiff
# import numpy as np
# import os
# import matplotlib.pyplot as plt
# from sklearn.metrics import precision_score, recall_score, f1_score
from PIL import Image
import rasterio
from sklearn.model_selection import train_test_split;
import re
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
import matplotlib.pyplot as plt
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score


import json

with open('config.json') as json_file:
    config = json.load(json_file)
    corruption = config['corruption_level']


c:\Users\ADMIN\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\ADMIN\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
def conv_block(x, filters, batchnorm=True):
    conv1 = Conv2D(filters, (3, 3), kernel_initializer='he_normal', padding='same')(x)
    if batchnorm is True:
        conv1 = BatchNormalization(axis=3)(conv1)
    conv1 = Activation('relu')(conv1)    
    conv2 = Conv2D(filters, (3, 3), kernel_initializer='he_normal', padding='same')(conv1)
    if batchnorm is True:
        conv2 = BatchNormalization(axis=3)(conv2)
    conv2 = Activation("relu")(conv2)

    return conv2

In [4]:
def residual_conv_block(x, filters, batchnorm=True):
    conv1 = Conv2D(filters, (3, 3), kernel_initializer='he_normal', padding='same')(x)
    if batchnorm is True:
        conv1 = BatchNormalization(axis=3)(conv1)
    conv1 = Activation('relu')(conv1)    
    conv2 = Conv2D(filters, (3, 3), kernel_initializer='he_normal', padding='same')(conv1)
    if batchnorm is True:
        conv2 = BatchNormalization(axis=3)(conv2)
    conv2 = Activation("relu")(conv2)
        
    #skip connection    
    shortcut = Conv2D(filters, kernel_size=(1, 1), kernel_initializer='he_normal', padding='same')(x)
    if batchnorm is True:
        shortcut = BatchNormalization(axis=3)(shortcut)
    shortcut = Activation("relu")(shortcut)
    respath = add([shortcut, conv2])       
    return respath

In [5]:
def dense_block(inputs, num_filters):
    conv1 = conv_block(inputs, num_filters)
    concat = Concatenate()([inputs, conv1])
    return concat

In [6]:
def residual_unet(input_shape):
    inputs = Input(input_shape)
    
    # Encoder
    conv1 = residual_conv_block(inputs, 64)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = residual_conv_block(pool1, 128)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = residual_conv_block(pool2, 256)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = residual_conv_block(pool3, 512)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    # Bottleneck
    conv5 = Conv2D(1024, 3, activation='relu', kernel_initializer='he_normal', padding='same')(pool4)
    conv5 = Conv2D(1024, (3, 3), kernel_initializer='he_normal', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)
    
    # Decoder
    up6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(drop5)
    up6 = Concatenate()([up6, conv4])
    conv6 = residual_conv_block(up6, 512)
    up7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
    up7 = Concatenate()([up7, conv3])
    conv7 = residual_conv_block(up7, 256)
    up8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
    up8 = Concatenate()([up8, conv2])
    conv8 = residual_conv_block(up8, 128)
    up9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
    up9 = Concatenate()([up9, conv1])
    conv9 = residual_conv_block(up9, 64)
    
    # Output
    outputs = Conv2D(1, 1, activation='sigmoid')(conv9)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [7]:
def dense_unet(input_shape):
    inputs = Input(input_shape)
    
    # Encoder
    conv1 = dense_block(inputs, 64)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = dense_block(pool1, 128)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = dense_block(pool2, 256)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = dense_block(pool3, 512)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    # Bottleneck
    conv5 = Conv2D(1024, 3, activation='relu', kernel_initializer='he_normal', padding='same')(pool4)
    conv5 = Conv2D(1024, (3, 3), kernel_initializer='he_normal', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)
    
    # Decoder
    up6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(drop5)
    up6 = Concatenate()([up6, conv4])
    conv6 = residual_conv_block(up6, 512)
    up7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
    up7 = Concatenate()([up7, conv3])
    conv7 = residual_conv_block(up7, 256)
    up8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
    up8 = Concatenate()([up8, conv2])
    conv8 = residual_conv_block(up8, 128)
    up9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
    up9 = Concatenate()([up9, conv1])
    conv9 = residual_conv_block(up9, 64)
    
    # Output
    outputs = Conv2D(1, 1, activation='sigmoid')(conv9)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [8]:
def load_data(input_dir, mask_dir):
    def numerical_sort(value):
        numbers = re.findall(r'\d+', os.path.basename(value))
        return int(numbers[-1]) if numbers else -1

    # Ensure the directories exist
    if not os.path.exists(input_dir):
        print(f"Input directory {input_dir} does not exist.")
        return None, None
    
    if not os.path.exists(mask_dir):
        print(f"Mask directory {mask_dir} does not exist.")
        return None, None

    # Search for .tif files in the directories and sort them numerically
    input_files = sorted(glob.glob(os.path.join(input_dir, '*.tif')), key=numerical_sort)
    mask_files = sorted(glob.glob(os.path.join(mask_dir, '*.tif')), key=numerical_sort)

    # print("Found input files:", input_files)
    # print("Found mask files:", mask_files)

    if not input_files:
        print(f"No input files found in {input_dir}.")
        return None, None

    if not mask_files:
        print(f"No mask files found in {mask_dir}.")
        return None, None

    images = []
    masks = []

    for input_file, mask_file in zip(input_files, mask_files):
        # Read input file
        with rasterio.open(input_file) as src:
            img = src.read(1)  # Read the first band assuming it's a single-band image
            # print(f"Reading image from {input_file}")
            images.append(img)

        # Read mask file
        with rasterio.open(mask_file) as src:
            msk = src.read(1)  # Read the first band assuming it's a single-band image
            # print(msk)
            masks.append(msk)

    return np.array(images), np.array(masks)


In [9]:
def load_data_new(input_dir, mask_dir):
    
    def numerical_sort(value):
        numbers = re.findall(r'\d+', os.path.basename(value))
        return int(numbers[-1]) if numbers else -1

    # Ensure the directories exist
    if not os.path.exists(input_dir):
        print(f"Input directory {input_dir} does not exist.")
        return None, None
    
    if not os.path.exists(mask_dir):
        print(f"Mask directory {mask_dir} does not exist.")
        return None, None

    # Search for .tif files in the directories
    input_files = glob.glob(os.path.join(input_dir, '*.tif'))
    mask_files = glob.glob(os.path.join(mask_dir, '*.tif'))

    # print("Found input files:", input_files)
    # print("Found mask files:", mask_files)

    if not input_files:
        print(f"No input files found in {input_dir}.")
        return None, None

    if not mask_files:
        print(f"No mask files found in {mask_dir}.")
        return None, None

    images = []
    masks = []

    for mask_file in mask_files:
        # Extract the number i from the mask filename
        match = re.search(r'NDWI_Mask_(\d+)_resized_corrupt.tif', os.path.basename(mask_file))
        print(match)
        if match:
            i = match.group(1)
            input_file = os.path.join(input_dir, f'{i}.tif')
            # print("Input File : ",input_file)
            # Check if the corresponding input file exists
            if os.path.exists(input_file):
                # Read input file
                with rasterio.open(input_file) as src:
                    img = src.read(1)  # Read the first band assuming it's a single-band image
                    images.append(img)

                # Read mask file
                with rasterio.open(mask_file) as src:
                    msk = src.read(1)  # Read the first band assuming it's a single-band image
                    masks.append(msk)
            else:
                print(f"Corresponding input file {input_file} for mask {mask_file} not found.")
        # break
    if not images or not masks:
        print("No matching pairs of images and masks found.")
        return None, None

    return np.array(images), np.array(masks)

In [10]:
# def load_data_test_new(input_dir, mask_dir):
    
#     def numerical_sort(value):
#         numbers = re.findall(r'\d+', os.path.basename(value))
#         return int(numbers[-1]) if numbers else -1

#     # Ensure the directories exist
#     if not os.path.exists(input_dir):
#         print(f"Input directory {input_dir} does not exist.")
#         return None, None
    
#     if not os.path.exists(mask_dir):
#         print(f"Mask directory {mask_dir} does not exist.")
#         return None, None

#     # Search for .tif files in the directories
#     input_files = glob.glob(os.path.join(input_dir, '*.tif'))
#     mask_files = glob.glob(os.path.join(mask_dir, '*.tif'))

#     print("Found input files:", input_files)
#     print("Found mask files:", mask_files)

#     if not input_files:
#         print(f"No input files found in {input_dir}.")
#         return None, None

#     if not mask_files:
#         print(f"No mask files found in {mask_dir}.")
#         return None, None

#     images = []
#     masks = []

#     for mask_file in mask_files:
#         # Extract the number i from the mask filename
#         match = re.search(r'NDWI_Mask_(\d+)_resized_corrupt.tif', os.path.basename(mask_file))
#         if match:
#             i = match.group(1)
#             input_file = os.path.join(input_dir, f'{i}.tif')
            
#             # Check if the corresponding input file exists
#             if os.path.exists(input_file):
#                 # Read input file
#                 with rasterio.open(input_file) as src:
#                     img = src.read(1)  # Read the first band assuming it's a single-band image
#                     images.append(img)

#                 # Read mask file
#                 with rasterio.open(mask_file) as src:
#                     msk = src.read(1)  # Read the first band assuming it's a single-band image
#                     masks.append(msk)
#             else:
#                 print(f"Corresponding input file {input_file} for mask {mask_file} not found.")

#     if not images or not masks:
#         print("No matching pairs of images and masks found.")
#         return None, None

#     return np.array(images), np.array(masks)

In [11]:
def load_data_test(input_dir, mask_dir):
    
    def numerical_sort(value):
        numbers = re.findall(r'\d+', os.path.basename(value))
        return int(numbers[-1]) if numbers else -1

    # Ensure the directories exist
    if not os.path.exists(input_dir):
        print(f"Input directory {input_dir} does not exist.")
        return None, None
    
    if not os.path.exists(mask_dir):
        print(f"Mask directory {mask_dir} does not exist.")
        return None, None

    # Search for .tif files in the directories
    input_files = glob.glob(os.path.join(input_dir, '*.tif'))
    mask_files = glob.glob(os.path.join(mask_dir, '*.tif'))

    # print("Found input files:", input_files)
    # print("Found mask files:", mask_files)

    if not input_files:
        print(f"No input files found in {input_dir}.")
        return None, None

    if not mask_files:
        print(f"No mask files found in {mask_dir}.")
        return None, None

    images = []
    masks = []

    for mask_file in mask_files:
        # Extract the number i from the mask filename
        match = re.search(r'NDWI_Mask_(\d+)_resized.tif', os.path.basename(mask_file))
        if match:
            i = match.group(1)
            input_file = os.path.join(input_dir, f'{i}.tif')
            
            # Check if the corresponding input file exists
            if os.path.exists(input_file):
                # Read input file
                with rasterio.open(input_file) as src:
                    img = src.read(1)  # Read the first band assuming it's a single-band image
                    images.append(img)

                # Read mask file
                with rasterio.open(mask_file) as src:
                    msk = src.read(1)  # Read the first band assuming it's a single-band image
                    masks.append(msk)
            else:
                print(f"Corresponding input file {input_file} for mask {mask_file} not found.")

    if not images or not masks:
        print("No matching pairs of images and masks found.")
        return None, None

    return np.array(images), np.array(masks)

In [12]:
# def load_data1(input_dir, mask_dir):
#     # Ensure the directories exist
#     if not os.path.exists(input_dir):
#         print(f"Input directory {input_dir} does not exist.")
#         return None, None
    
#     if not os.path.exists(mask_dir):
#         print(f"Mask directory {mask_dir} does not exist.")
#         return None, None

#     # Print the contents of the directories
#     print("Input directory contents:", os.listdir(input_dir))
#     print("Mask directory contents:", os.listdir(mask_dir))

#     # Search for .tif files in the directories
#     input_files = sorted(glob.glob(os.path.join(input_dir, '*.tif')))
#     mask_files = sorted(glob.glob(os.path.join(mask_dir, '*.tif')))

#     print("Found input files:", input_files)
#     print("Found mask files:", mask_files)

#     if not input_files:
#         print(f"No input files found in {input_dir}.")
#         return None, None

#     if not mask_files:
#         print(f"No mask files found in {mask_dir}.")
#         return None, None

#     images = []
#     masks = []

#     for input_file, mask_file in zip(input_files, mask_files):
#         # Read input file
#         with rasterio.open(input_file) as src:
#             img = src.read(1)  # Read the first band assuming it's a single-band image
#             print(f"Reading image from {input_file}")
#             images.append(img)

#         # Read mask file
#         with rasterio.open(mask_file) as src:
#             msk = src.read(1)  # Read the first band assuming it's a single-band image
#             masks.append(msk)

    # return np.array(images), np.array(masks) / 255

In [13]:
def load_data_png(input_dir, mask_dir):
    input_files = sorted(glob.glob(os.path.join(input_dir, '*.png')))
    # print(input_files)
    mask_files = sorted(glob.glob(os.path.join(mask_dir, '*.png')))
    # print(mask_files)
    images = []
    masks = []

    for input_file, mask_file in zip(input_files, mask_files):

        image = cv2.imread(input_file)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        images.append(np.array(image_rgb))

        image = cv2.imread(mask_file)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        masks.append(np.array(image_rgb))

    # print(np.array(masks)/255)
    return np.array(images), np.array(masks)/255

In [14]:
def predict(model, images, masks, output_dir, model_type, number=-1):

    i=0
    for arbitrary_img, arbitrary_mask in zip(images, masks):

    # arbitrary_image_path = 'data_new\subset_0_of_subset_1_of_final_S1A_IW_GRDH_1SDV_20240109T095534_20240109T095559_052028_06499B_56C7_0_1024.tif'
    # arbitrary_image_mask = 'output_tif\subset_0_of_subset_1_of_final_S1A_IW_GRDH_1SDV_20240109T095534_20240109T095559_052028_06499B_56C7_0_1024_new_mask.tif'

    # # Read the arbitrary image
    # with rasterio.open(arbitrary_image_path) as src:
    #     arbitrary_img = src.read(1)  # Read the first band assuming it's a single-band image

    # with rasterio.open(arbitrary_image_mask) as src:
    #     arbitrary_mask = src.read(1)

    # Preprocess the arbitrary image if necessary (e.g., normalize, resize) to match the input dimensions of your model
    # ...

        predicted_mask = model.predict(np.expand_dims(arbitrary_img, axis=0))[0]

        # Plot the arbitrary image and the predicted mask
        plt.figure(figsize=(10, 5))

        plt.subplot(2, 2, 1)
        plt.imshow(arbitrary_img, cmap='gray')
        plt.title('Arbitrary Image')
        plt.axis('off')

        plt.subplot(2, 2, 2)
        # print(arbitrary_mask)
        print(np.min(arbitrary_mask))
        print(np.max(arbitrary_mask))
        plt.imshow(arbitrary_mask, cmap='gray')
        plt.title('Actual Mask')
        plt.axis('off')

        plt.subplot(2, 2, 3)
        plt.imshow(predicted_mask, cmap='gray')
        print("Predicted_Mask")
        print("Min value in predicted_mask:", np.min(predicted_mask))
        print("Max value in predicted_mask:", np.max(predicted_mask))
        plt.title('Predicted Mask without thresholding')
        plt.axis('off')

         # 
         #hold the predicted mask if necessary
        predicted_mask = (predicted_mask > 0.70).astype(np.uint8)

        plt.subplot(2, 2, 4)
        plt.imshow(predicted_mask, cmap='gray')
        plt.title('Predicted Mask')
        plt.axis('off')

        plt.savefig(f"./{output_dir}/{model_type}_{i}.jpg")
        plt.close()
        i+=1
        # break
        if (i==number):
            break

In [15]:
def dice_coefficient(y_true, y_pred, smooth=1):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

def iou(y_true, y_pred, smooth=1):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    union = np.sum(y_true_f) + np.sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)

def save_image(image, filepath):
    image = (image * 255).astype(np.uint8)  # Scale image to 0-255
    tiff.imwrite(filepath, image)

def predict1(model, mask_dir, image_dir, output_dir, model_type, number=-1):
    dice_scores = []
    iou_scores = []
    precisions = []
    recalls = []
    f1_scores = []

    os.makedirs(output_dir, exist_ok=True)

    mask_files = [f for f in os.listdir(mask_dir) if f.endswith('_resized.tif')]
    i = 0
    print(len(mask_files))
    for mask_file in mask_files:
        # try:
        # Extract the number i from the mask file name
        print(mask_file)
        i_str = mask_file.split('_')[2]
        print(i_str)
        image_file = f"{i_str}.tif"

        mask_path = os.path.join(mask_dir, mask_file)
        image_path=os.path.join(image_dir, image_file)
        print(image_path)

        # Load the image and mask
        arbitrary_img = tiff.imread(image_path)
        arbitrary_mask = tiff.imread(mask_path)

        # Ensure the image is in the correct shape (512, 512, 1)
        if len(arbitrary_img.shape) == 2:
            arbitrary_img = np.expand_dims(arbitrary_img, axis=-1)
        elif arbitrary_img.shape[0] == 2:
            arbitrary_img = arbitrary_img[0]  # assuming you need the first channel
        
        predicted_mask = model.predict(np.expand_dims(arbitrary_img, axis=0))[0]

        # Apply thresholding to predicted mask
        predicted_mask_thresh = (predicted_mask > 0.5).astype(np.uint8)

        # Save predicted mask images
        save_image(predicted_mask, f"./{output_dir}/{model_type}_Predicted_Image_{i_str}.tif")
        save_image(predicted_mask_thresh, f"./{output_dir}/{model_type}_{i_str}.tif")

        # Plot and save the arbitrary image, actual mask, and predicted mask
        plt.figure(figsize=(10, 5))

        plt.subplot(2, 2, 1)
        plt.imshow(arbitrary_img.squeeze(), cmap='gray')
        plt.title('Arbitrary Image')
        plt.axis('off')

        plt.subplot(2, 2, 2)
        plt.imshow(arbitrary_mask, cmap='gray')
        plt.title('Actual Mask')
        plt.axis('off')

        plt.subplot(2, 2, 3)
        plt.imshow(predicted_mask.squeeze(), cmap='gray')
        plt.title('Predicted Mask without thresholding')
        plt.axis('off')

        plt.subplot(2, 2, 4)
        plt.imshow(predicted_mask_thresh.squeeze(), cmap='gray')
        plt.title('Predicted Mask')
        plt.axis('off')

        plt.savefig(f"./{output_dir}/{model_type}_Visualization_{i_str}.jpg")
        plt.close()

        # Calculate metrics
        dice = dice_coefficient(arbitrary_mask, predicted_mask_thresh)
        iou_score = iou(arbitrary_mask, predicted_mask_thresh)
        precision = precision_score(arbitrary_mask.flatten(), predicted_mask_thresh.flatten())
        recall = recall_score(arbitrary_mask.flatten(), predicted_mask_thresh.flatten())
        f1 = f1_score(arbitrary_mask.flatten(), predicted_mask_thresh.flatten())

        dice_scores.append(dice)
        iou_scores.append(iou_score)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)
    
        i += 1
        if i == number:
            break
    # Print and return the average metrics
    mean_dice = np.mean(dice_scores)
    mean_iou = np.mean(iou_scores)
    mean_precision = np.mean(precisions)
    mean_recall = np.mean(recalls)
    mean_f1 = np.mean(f1_scores)

    print(f"Mean Dice Coefficient: {mean_dice}")
    print(f"Mean IoU: {mean_iou}")
    print(f"Mean Precision: {mean_precision}")
    print(f"Mean Recall: {mean_recall}")
    print(f"Mean F1 Score: {mean_f1}")

    return mean_dice, mean_iou, mean_precision, mean_recall, mean_f1

In [16]:
input_dir = './data_new/'
mask_dir = f'./GEE_Masks/GEE_resized/train_gee/train_{corruption}_gee_with_diff_kernels'
output_dir = f'./GEE_Output/Adversarial/New_20_Epoch_{corruption}_with_diff_kernels'

print(mask_dir)
print(output_dir)

os.makedirs(output_dir,exist_ok=True)
# Load data
images, masks = load_data_new(input_dir, mask_dir)

# Define input shape and number of classes
images = np.expand_dims(images, axis=-1)
input_shape = images.shape[1:]

# images_train, images_test, masks_train, masks_test = train_test_split(images, masks, test_size=0.2, random_state=42)

# Create and compile the model

# for layer in model.layers:
#     print(layer.output_shape)


./GEE_Masks/GEE_resized/train_gee/train_25_gee_with_diff_kernels
./GEE_Output/Adversarial/New_20_Epoch_25_with_diff_kernels
<re.Match object; span=(0, 31), match='NDWI_Mask_0_resized_corrupt.tif'>
<re.Match object; span=(0, 34), match='NDWI_Mask_1000_resized_corrupt.tif'>
<re.Match object; span=(0, 34), match='NDWI_Mask_1001_resized_corrupt.tif'>
<re.Match object; span=(0, 34), match='NDWI_Mask_1002_resized_corrupt.tif'>
<re.Match object; span=(0, 34), match='NDWI_Mask_1003_resized_corrupt.tif'>
<re.Match object; span=(0, 34), match='NDWI_Mask_1004_resized_corrupt.tif'>
<re.Match object; span=(0, 34), match='NDWI_Mask_1005_resized_corrupt.tif'>
<re.Match object; span=(0, 34), match='NDWI_Mask_1006_resized_corrupt.tif'>
<re.Match object; span=(0, 34), match='NDWI_Mask_1007_resized_corrupt.tif'>


c:\Users\ADMIN\anaconda3\lib\site-packages\rasterio\__init__.py:333: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


<re.Match object; span=(0, 34), match='NDWI_Mask_1009_resized_corrupt.tif'>
<re.Match object; span=(0, 33), match='NDWI_Mask_100_resized_corrupt.tif'>
<re.Match object; span=(0, 34), match='NDWI_Mask_1011_resized_corrupt.tif'>
<re.Match object; span=(0, 34), match='NDWI_Mask_1013_resized_corrupt.tif'>
<re.Match object; span=(0, 34), match='NDWI_Mask_1014_resized_corrupt.tif'>
<re.Match object; span=(0, 34), match='NDWI_Mask_1015_resized_corrupt.tif'>
<re.Match object; span=(0, 34), match='NDWI_Mask_1016_resized_corrupt.tif'>
<re.Match object; span=(0, 34), match='NDWI_Mask_1017_resized_corrupt.tif'>
<re.Match object; span=(0, 34), match='NDWI_Mask_1018_resized_corrupt.tif'>
<re.Match object; span=(0, 34), match='NDWI_Mask_1019_resized_corrupt.tif'>
<re.Match object; span=(0, 33), match='NDWI_Mask_101_resized_corrupt.tif'>
<re.Match object; span=(0, 34), match='NDWI_Mask_1021_resized_corrupt.tif'>
<re.Match object; span=(0, 34), match='NDWI_Mask_1022_resized_corrupt.tif'>
<re.Match obje

In [17]:
input_shape = images.shape[1:]
print(input_shape)
print(len(images))

(512, 512, 1)
1010


In [ ]:
model = dense_unet(input_shape)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Train the model
model.fit(images, masks, validation_split=0.125, epochs=20, batch_size=2)  # Adjust epochs and batch size as needed


Epoch 1/20
442/442 [==============================] - 287s 578ms/step - loss: 0.4559 - accuracy: 0.8063 - val_loss: 0.3154 - val_accuracy: 0.8323
Epoch 2/20
442/442 [==============================] - 242s 548ms/step - loss: 0.4344 - accuracy: 0.8137 - val_loss: 0.3893 - val_accuracy: 0.8862
Epoch 3/20
442/442 [==============================] - 242s 548ms/step - loss: 0.4268 - accuracy: 0.8181 - val_loss: 0.2149 - val_accuracy: 0.9462
Epoch 4/20
442/442 [==============================] - 242s 547ms/step - loss: 0.4257 - accuracy: 0.8200 - val_loss: 0.2512 - val_accuracy: 0.8580
Epoch 5/20
442/442 [==============================] - ETA: 0s - loss: 0.4195 - accuracy: 0.8219

In [19]:
model_name = f'UNet_dense_GEE_20_epoch_{corruption}_with_diff_kernels'
print(model_name)
model.save(model_name)

UNet_dense_GEE_20_epoch_30_with_diff_kernels


INFO:tensorflow:Assets written to: UNet_dense_GEE_20_epoch_30_with_diff_kernels\assets


INFO:tensorflow:Assets written to: UNet_dense_GEE_20_epoch_30_with_diff_kernels\assets


In [20]:
# import matplotlib.pyplot as plt

# # Data from the model training output
# epochs = range(1, 21)
# train_loss = [0.2544, 0.2217, 0.2159, 0.2108, 0.2045, 0.2043, 0.2045, 0.1992, 0.2013, 0.1996, 
#               0.1996, 0.1983, 0.1940, 0.1921, 0.1951, 0.1924, 0.1914, 0.1929, 0.1924, 0.1969]
# val_loss = [0.2999, 0.2028, 0.1811, 0.1712, 0.2268, 0.1686, 0.1759, 0.1667, 0.2325, 0.1723, 
#             0.1701, 0.1861, 0.1824, 0.1666, 0.1849, 0.1680, 0.1715, 0.1720, 0.1698, 0.2078]
# train_accuracy = [0.9185, 0.9285, 0.9303, 0.9313, 0.9328, 0.9308, 0.9328, 0.9336, 0.9331, 0.9340, 
#                  0.9331, 0.9333, 0.9354, 0.9351, 0.9346, 0.9350, 0.9352, 0.9352, 0.9357, 0.9324]
# val_accuracy = [0.8396, 0.9370, 0.9417, 0.9456, 0.9119, 0.9449, 0.9428, 0.9462, 0.9308, 0.9429, 
#                0.9462, 0.9429, 0.9363, 0.9462, 0.9398, 0.9461, 0.9426, 0.9443, 0.9454, 0.9354]

# # Plotting the training and validation loss
# plt.figure(figsize=(14, 6))

# plt.subplot(1, 2, 1)
# plt.plot(epochs, train_loss, label='Training Loss')
# plt.plot(epochs, val_loss, label='Validation Loss')
# plt.title('Training and Validation Loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()

# # Plotting the training and validation accuracy
# plt.subplot(1, 2, 2)
# plt.plot(epochs, train_accuracy, label='Training Accuracy')
# plt.plot(epochs, val_accuracy, label='Validation Accuracy')
# plt.title('Training and Validation Accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend()

# # Display the plots
# plt.tight_layout()
# plt.show()


In [21]:
# %%capture
# model = load_model('UNet_dense_10_epoch.h5')
# predict1(model, images, masks, output_dir, 'dense')

# import h5p
# input_dir = './data_new/'
mask_dir = './GEE_Masks/GEE_resized/test_gee/'
# output_dir = './GEE_Output/Adversarial/20_Epoch_Small'

# Load data
# images, masks = load_data_test(input_dir, mask_dir)
# model = load_model()

In [22]:
# model_name = 'UNet_dense_GEE_20_epoch_22_with_diff_kernels'
model = tf.keras.models.load_model(model_name, compile=False)
model.compile()
predict1(model,mask_dir,input_dir, output_dir, 'dense')

253
NDWI_Mask_1008_resized.tif
1008
./data_new/1008.tif
1/1 [==============================] - 0s 284ms/step
NDWI_Mask_1010_resized.tif
1010
./data_new/1010.tif
1/1 [==============================] - 0s 16ms/step
NDWI_Mask_1012_resized.tif
1012
./data_new/1012.tif
1/1 [==============================] - 0s 17ms/step
NDWI_Mask_1020_resized.tif
1020
./data_new/1020.tif
1/1 [==============================] - 0s 17ms/step
NDWI_Mask_1025_resized.tif
1025
./data_new/1025.tif
1/1 [==============================] - 0s 8ms/step
NDWI_Mask_1028_resized.tif
1028
./data_new/1028.tif
1/1 [==============================] - 0s 17ms/step
NDWI_Mask_1030_resized.tif
1030
./data_new/1030.tif
1/1 [==============================] - 0s 17ms/step
NDWI_Mask_1031_resized.tif
1031
./data_new/1031.tif
1/1 [==============================] - 0s 17ms/step
NDWI_Mask_1035_resized.tif
1035
./data_new/1035.tif
1/1 [==============================] - 0s 17ms/step
NDWI_Mask_1037_resized.tif
1037
./data_new/1037.tif
1/1 [===

c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


NDWI_Mask_1043_resized.tif
1043
./data_new/1043.tif
1/1 [==============================] - 0s 17ms/step
NDWI_Mask_1044_resized.tif
1044
./data_new/1044.tif
1/1 [==============================] - 0s 17ms/step


c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


NDWI_Mask_1048_resized.tif
1048
./data_new/1048.tif
1/1 [==============================] - 0s 17ms/step
NDWI_Mask_1058_resized.tif
1058
./data_new/1058.tif
1/1 [==============================] - 0s 17ms/step


...

c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1760: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


NDWI_Mask_960_resized.tif
960
./data_new/960.tif
1/1 [==============================] - 0s 17ms/step


c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


NDWI_Mask_974_resized.tif
974
./data_new/974.tif
1/1 [==============================] - 0s 17ms/step
NDWI_Mask_983_resized.tif
983
./data_new/983.tif
1/1 [==============================] - 0s 10ms/step
NDWI_Mask_986_resized.tif
986
./data_new/986.tif
1/1 [==============================] - 0s 17ms/step
NDWI_Mask_9_resized.tif
9
./data_new/9.tif
1/1 [==============================] - 0s 9ms/step
Mean Dice Coefficient: 0.25315114210268336
Mean IoU: 0.18224761880147097
Mean Precision: 0.7415656653616483
Mean Recall: 0.10418576446615954
Mean F1 Score: 0.17275713724045125


(0.25315114210268336,
 0.18224761880147097,
 0.7415656653616483,
 0.10418576446615954,
 0.17275713724045125)